In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [25]:
# Define the neural network architecture
class DependencyParser(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(DependencyParser, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, bidirectional=True)
        self.hidden2label = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, sentence):
        embeds = self.embedding(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        logits = self.hidden2label(lstm_out.view(len(sentence), -1))
        return logits

In [26]:
# Function to load and preprocess the data
def load_data(file_path):
    sentences = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        sentence = []
        label = []
        for line in file:
            line = line.strip()
            if line == '' or line.startswith('#'):  # Skip empty lines and comments
                continue
            parts = line.split('\t')
            word = parts[1]  # Word
            pos = parts[3]   # Part-of-speech tag
            if parts[6] != '_':  # Check if head index is available
                head = int(parts[6]) - 1  # Head of the current word
            else:
                head = -1  # Assign a default value when head index is missing
            label_type = parts[7]      # Dependency label
            sentence.append((word, pos))
            label.append(head)
            if parts[0] == '1':  # Start of a new sentence
                sentences.append(sentence)
                labels.append(label)
                sentence = []
                label = []
    return sentences, labels


In [27]:
# Function to create word-to-index and label-to-index mappings
def create_mappings(sentences, labels):
    word_to_idx = {}
    pos_to_idx = {}
    label_to_idx = {}
    for sentence, label in zip(sentences, labels):
        for word, pos in sentence:
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)
            if pos not in pos_to_idx:
                pos_to_idx[pos] = len(pos_to_idx)
        for head in label:
            if head not in label_to_idx:
                label_to_idx[head] = len(label_to_idx)
    return word_to_idx, pos_to_idx, label_to_idx

# Convert sentences to index sequences
def prepare_sequence(sentence, word_to_idx, pos_to_idx):
    word_indices = [word_to_idx.get(word, 0) for word, _ in sentence]
    pos_indices = [pos_to_idx[pos] for _, pos in sentence]
    return torch.tensor(word_indices).unsqueeze(0), torch.tensor(pos_indices).unsqueeze(0)


In [49]:
# Function to train the model
def train_model(model, train_data, train_labels, word_to_idx, pos_to_idx, label_to_idx, max_length, num_epochs=10, learning_rate=0.001):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss(ignore_index=-1)  # Ignore target values equal to -1

    for epoch in range(num_epochs):
        total_loss = 0
        for i in range(len(train_data)):
            model.zero_grad()
            sentence_in, pos_in = prepare_sequence(train_data[i], word_to_idx, pos_to_idx)
            targets = torch.tensor([label if label != -1 else 0 for label in train_labels[i]], dtype=torch.long)  # Replace -1 targets with 0
            targets = targets[:max_length]  # Ensure target length matches input length
            logits = model(sentence_in)
            loss = loss_function(logits.view(-1, logits.shape[-1]), targets.view(-1))  # Flatten logits and targets for consistency
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print("Epoch {}, Loss: {:.4f}".format(epoch+1, total_loss / len(train_data)))






In [29]:
# Function to evaluate the model
def evaluate_model(model, test_data, test_labels, word_to_idx, pos_to_idx):
    correct = 0
    total = 0
    with torch.no_grad():
        for sentence, labels in zip(test_data, test_labels):
            sentence_in, pos_in = prepare_sequence(sentence, word_to_idx, pos_to_idx)
            targets = torch.tensor(labels, dtype=torch.long)
            logits = model(sentence_in)
            predicted = torch.argmax(logits, dim=1)
            correct += torch.sum(predicted == targets).item()
            total += len(labels)
    accuracy = correct / total
    print("Accuracy: {:.2f}%".format(accuracy * 100))

In [30]:
# Load and preprocess the data
train_sentences, train_labels = load_data('en_ewt-ud-train.conllu')
test_sentences, test_labels = load_data('en_ewt-ud-test.conllu')

word_to_idx, pos_to_idx, label_to_idx = create_mappings(train_sentences, train_labels)
num_words = len(word_to_idx)
num_pos = len(pos_to_idx)
num_labels = len(label_to_idx)


In [40]:
# Initialize the model
input_dim = num_words
hidden_dim = 128
num_classes = num_labels
model = DependencyParser(input_dim, hidden_dim, num_classes)
# Define max_length based on the maximum length of sentences in your dataset
max_length = max(len(sentence) for sentence in train_sentences)


In [50]:
# Train the model
train_model(model, train_sentences, train_labels, word_to_idx, pos_to_idx, label_to_idx, max_length)


ValueError: Expected input batch_size (1) to match target batch_size (29).

In [ ]:
# Evaluate the model
evaluate_model(model, test_sentences, test_labels, word_to_idx, pos_to_idx)